In [1]:
import numpy as np
import pandas as pd
import string
import gensim
import operator
import re
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/kanunum-nlp-doc-analysis-dataset.csv')
data

,id,kategori,baslik,rega_no,mukerrer_no,rega_tarihi,kurum,mevzuat_no,belge_sayi,mevzuat_tarihi,donem,sira_no,madde_sayisi,data_text,url,kanunum_url
0,3,Kanun,BÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İL...,28350,0.0,2012-07-11,NaN,6338,NaN,2012-06-29,NaN,NaN,19,Resmî Gazete Tarihi: 11.07.2012 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2012/07...,https://www.kanunum.com/content/344634
1,5,Kanun,SAĞLIK BAKANLIĞI VE BAĞLI KURULUŞLARININ TEŞKİ...,28886,0.0,2014-01-18,NaN,6514,NaN,2014-01-02,NaN,NaN,56,Resmî Gazete Tarihi: 18.01.2014 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2014/01...,https://www.kanunum.com/content/942558
2,9,Kanun,İŞ KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KAR...,29116,1.0,2014-09-11,NaN,6552,NaN,2014-09-10,NaN,NaN,149,Resmî Gazete Tarihi: 11.09.2014 Resmî Gazete S...,https://www.mevzuat.gov.tr/mevzuat?MevzuatNo=6...,https://www.kanunum.com/content/1233870
3,14,Kanun,YÜKSEKÖĞRETİM KURUMLARI TEŞKİLATI KANUNU İLE ...,29335,0.0,2015-04-23,NaN,6640,NaN,2015-03-31,NaN,NaN,11,Resmî Gazete Tarihi: 23.04.2015 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2015/04...,https://www.kanunum.com/content/1495629
4,15,Kanun,ASKERLİK KANUNU VE BAZI KANUNLARDA DEĞİŞİKLİK ...,29606,0.0,2016-01-27,NaN,6661,NaN,2016-01-14,NaN,NaN,21,Resmî Gazete Tarihi: 27.01.2016 Resmî Gazete S...,https://www.resmigazete.gov.tr/eskiler/2016/01...,https://www.kanunum.com/content/2414283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137,8843,Özelge,"T.C.GELİR İDARESİ BAŞKANLIĞI, Kooperatif üyele...",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,NaN,NaN,2011-08-26,NaN,NaN,0,BaşlıkKooperatif üyelerinin 38 adet dükkan ve ...,http://www.gib.gov.tr/index.php?id=1031&no_cac...,https://www.kanunum.com/content/178911
4138,8844,Özelge,"T.C.GELİR İDARESİ BAŞKANLIĞI, Engellilerin oto...",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,NaN,NaN,2011-10-28,NaN,NaN,0,BaşlıkEngellilerin otomobil alımlarında KDV is...,http://www.gib.gov.tr/index.php?id=1031&no_cac...,https://www.kanunum.com/content/642868
4139,8846,Özelge,"T.C.GELİR İDARESİ BAŞKANLIĞI, İsviçre mukimi f...",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,NaN,NaN,2015-03-24,NaN,NaN,0,T.C. \n\nGELİR İDARESİ BAŞKANLIĞI \n\nİSTANBUL...,http://www.gib.gov.tr/node/110626,https://www.kanunum.com/content/2645962
4140,8849,Özelge,"T.C.GELİR İDARESİ BAŞKANLIĞI, Kolektif şirket ...",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,NaN,NaN,2015-06-18,NaN,NaN,0,T.C. \n\nGELİR İDARESİ BAŞKANLIĞI \n\nİSTANBUL...,http://www.gib.gov.tr/node/105737,https://www.kanunum.com/content/1629642


In [ ]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df=pd.DataFrame(data["kategori"].astype(str))
print(df.kategori.unique())
df['labels'] = pd.factorize(df.kategori)[0]

['Kanun' 'Resmi Gazete' 'Kanun Hükmünde Kararname' 'Genelge'
 'Komisyon Raporu' 'Cumhurbaşkanlığı Kararnamesi' 'Tüzük' 'Yönetmelik'
 'Tebliğ' 'Özelge']


In [ ]:
pip install simpletransformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=10, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":True, "output_dir": "bert_model"})

model.train_model(train)
result, model_outputs, wrong_predictions = model.eval_model(test)

predictions = model_outputs.argmax(axis=1)

from sklearn.metrics import accuracy_score
actuals = test.labels.values
accuracy_score(actuals, predictions)


from sklearn import metrics
print(metrics.classification_report(actuals, predictions, digits=3))

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

  0%|          | 0/3313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/52 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/52 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/52 [00:00<?, ?it/s]